In [1]:
from ovcg import *
from fesol import *
from pyccx import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper

In [2]:
k=5
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)

use_relax = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD%i.pvd' % k)
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid solver
jobname = 'solid/solidD'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=True, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=False, itype=CHTAdapter.DFLUX)

# now we are ready

In [5]:
Tf = 'Tf'
Ts = 'Ts'
Ff = 'Ff'
Fs = 'Fs'
Ff2Fs = {Tf:Ts}
Ts2Tf = {Fs:Ff}
f2s_mapper, s2f_mapper = generate_mapper(fnodes, [snodes,scents], Ff2Fs, Ts2Tf,r1=0.05,r2=0.2)

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=fnodes.shape[0], value=0.0)
fluxS = InterfaceData(size=scents.shape[0], value=0.0)
tempF = InterfaceData(size=fnodes.shape[0], value=300.0)
tempS = InterfaceData(size=snodes.shape[0], value=310.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [ ]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 500
avg_pc_iterations = 0.0
flog = open('FNSD_avg%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD_avg%i.log' % k, mode='a')

temps_prev = tempS.curr.copy()
fluxf_prev = fluxF.curr.copy()

In [ ]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup_state()
    
    temps_prev[:] = tempS.curr
    fluxf_prev[:] = fluxF.curr
    
    # mapping
    s2f_mapper.put(Fs, fluxS.curr)
    s2f_mapper.apply(Fs, Ff)
    fluxF.curr[:] = s2f_mapper.get(Ff)
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=fluxF.curr
    )
    
    while True:
        # back up previous interface value
        fluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        f2s_mapper.put(Tf, tempF.curr)
        f2s_mapper.apply(Tf, Ts)
        tempS.curr[:] = f2s_mapper.get(Ts)
        # update solid flux interface
        solverS['Inodes', SET].set_temperatures(0.5*(tempS.curr+temps_prev))
        
        # advance solid
        solverS.adjust_timesize(dt)
        solverS.increment()
        # retrieve solid interface temperature
        fluxS.curr[:] = solverS['Interface', GET].get_dfluxes()
        
        # mapping
        s2f_mapper.put(Fs, fluxS.curr)
        s2f_mapper.apply(Fs, Ff)
        fluxF.curr[:] = s2f_mapper.get(Ff)
        
        # update residual
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.finish_increment()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(fluxF)
                under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(fluxF.curr+fluxf_prev)
            )
            solverF.restore_previous_solutions()
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

step=0, pc_iterations=10.
step=1, pc_iterations=9.
step=2, pc_iterations=9.
step=3, pc_iterations=8.
step=4, pc_iterations=9.
step=5, pc_iterations=9.
step=6, pc_iterations=8.
step=7, pc_iterations=8.


In [ ]:
theta=(solverF.get_temperature_interface(3,0,1)[0:100]-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFNSD_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)